# Clasificador Bayesiano


### Librerías a utilizar:

- e1071
- caret

### ¿Qué es un clasificador? ¿Bayesiano?
Un clasificador corresponde a un modelo de aprendizaje automático que es capaz de reconocer patrones respecto a un valor objetivo. Para esta sesión utilizaremos y estudiaremos el clasificador bayesiano ingenuo, el cual se basa en la probabilidad condicional para "aprender" a clasificar.

Un recordatorio, ¿qué es el teorema de bayes?

> El teorema de Bayes, en la teoría de la probabilidad, es una proposición planteada por el matemático inglés Thomas Bayes (1702-1761)1​ y publicada póstumamente en 1763,2​ que expresa la probabilidad condicional de un evento aleatorio A dado B en términos de la distribución de probabilidad condicional del evento B dado A y la distribución de probabilidad marginal de solo A. (Wikipedia)

$$ p(A|B) = \frac{p(A \cap B)}{p(B)} = \frac{p(A)p(B|A)}{p(B)} $$

Por lo tanto, para crear un clasificador bayesiano solo hace falta conocer las probabilidades condicionales de las caracteristicas en conjunto a la ocurrencia de la clase.

$$ Clase_{NBC} = arg max \{ p(c_i) \Pi^p_i p(a_j / c_i)) \} $$

- $p(c_i)$ = probabilidad a priori
- $p(a_j / c_i)$ = probabilidad a posteriori

Cuando en la probabilidad a posteriori se tienen numeros reales, se calcula:

$$ p(a_j/c_i) = \frac{1}{\sqrt(2 \cdot \pi) \cdot \sigma_j } \cdot e^{\frac{(a_j - \bar{a_j})^2}{2 \cdot \sigma_j}}$$


Si quisieramos evaluar la diabetes Mellitus, en donde supuestamente existen dos variables de interés, el nivel sobrepeso y el nivel de glucosa en la sangre del paciente. De un hospital se tiene la siguiente tabla:

|  			Nivel de sobrepeso 		 |  			Glucosa en mg/100 ml de sangre 		 |  			Diabético 		 |
|:--------------------:|:--------------------------------:|:-----------:|
|        			Normal 		       |                			70 		               |      			No 		    |
|        			Normal 		       |                			80 		               |      			No 		    |
|       			Sobrepeso 		     |                			90 		               |      			No 		    |
|        			Normal 		       |                			100 		              |      			No 		    |
|         			Obeso 		       |                			110 		              |      			No 		    |
|       			Sobrepeso 		     |                			120 		              |      			Sí 		    |
|       			Sobrepeso 		     |                			130 		              |      			Sí 		    |
|         			Obeso 		       |                			140 		              |      			Sí 		    |
|         			Obeso 		       |                			150 		              |      			Sí 		    |



Tenemos que la clase corresponde a Diabetes={si, no}, por lo tanto la probabilidad a priori corresponde a:

- p(Diabetes=si) = 4/9
- p(Diabetes=no) = 5/9

Luego se deben calcular las probabilidades a posteriori:

- p(Nivel de sobrepeso = Normal / Diabetes=si) = $\frac{0/9}{4/9}$ = 0
- p(Nivel de sobrepeso = Sobrepeso / Diabetes=si) = $\frac{2/9}{4/9}$ = 1/2
- p(Nivel de sobrepeso = Obeso / Diabetes=si) = $\frac{2/9}{4/9}$ = 1/2


- p(Nivel de sobrepeso = Normal / Diabetes=no) = $\frac{3/9}{5/9}$ = 3/5
- p(Nivel de sobrepeso = Sobrepeso / Diabetes=no) = $\frac{1/9}{5/9}$ = 1/5
- p(Nivel de sobrepeso = Obeso / Diabetes=no) = $\frac{1/9}{5/9}$ = 1/5

Ademas:

- $\sigma_{Glucosa\_no} = 15.8$ y $\bar{Glucosa\_no} = 90$  
- $\sigma_{Glucosa\_si} = 12.9$ y $\bar{Glucosa\_si} = 135$

Luego, se tiene constituido el clasificador tipo:

$$ Clase_{NBC} = arg max \{ p(c_i) \Pi^p_i p(a_j / c_i)) \} $$

Si llega un individuo con **Glucosa=90 mg/mL** y su **Nivel de sobrepeso = Obeso**, se tiene:

$$ Clase_{NBC} = arg max \{ \\ 
4/9 \cdot (\text{p(NS = Obeso / Diabetes=si)} \cdot \text{p(NG = 90 / Diabetes=si}), \\ 5/9 \cdot (\text{p(NS = Obeso / Diabetes=no)} \cdot \text{p(NG = 90 / Diabetes=no}) \\ \} $$


Quedando:

$$ Clase_{NBC} = arg max \{ 0.0000155 , 0.0028 \} $$

Por lo tanto el nuevo paciente no tiene diabetes segun el clasificador bayesiano.

In [2]:
library("e1071")
library("caret")

In [5]:
example = data.frame(
    ns=c("Normal", "Normal", "Sobrepeso", "Normal", "Obeso", "Sobrepeso", "Sobrepeso", "Obeso", "Obeso"),
    ng=c(70, 80, 90, 100, 110, 120, 130, 140, 150),
    d=c("no","no","no","no","no","si","si","si","si")
)

In [6]:
example

ns,ng,d
Normal,70,no
Normal,80,no
Sobrepeso,90,no
Normal,100,no
Obeso,110,no
Sobrepeso,120,si
Sobrepeso,130,si
Obeso,140,si
Obeso,150,si


In [7]:
bayesian.classifier = naiveBayes(d ~ ., data = example)
bayesian.classifier


Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
       no        si 
0.5555556 0.4444444 

Conditional probabilities:
    ns
Y    Normal Obeso Sobrepeso
  no    0.6   0.2       0.2
  si    0.0   0.5       0.5

    ng
Y    [,1]     [,2]
  no   90 15.81139
  si  135 12.90994


In [14]:
pacients = data.frame(ns=c("Obeso", "Sobrepeso"), ng=c(90, 150))
response = predict(object = bayesian.classifier, pacients)

In [15]:
response

[1] no si
Levels: no si

In [51]:
columns = c("area", "perimeter", "compactness", "length", "width", "AC", "lengthGroove", "class")
url = "https://www.dl.dropboxusercontent.com/s/wrexlo5im3g5ioi/seeds_dataset.csv"
seeds = read.csv(url, header = F, sep=",", col.names = columns)
seeds$class = factor(seeds$class, levels = c(1,2,3), labels = c("Kama", "Rosa", "Canadian"))
seeds = seeds[c(1, 4, 6, 8)]

In [52]:
set.seed(1313)
training.index = createDataPartition(seeds$class, p=0.7)$Resample1
training.set = seeds[training.index, ]
test.set = seeds[-training.index, ]

In [53]:
bayesian.classifier = naiveBayes(class ~ ., data = training.set)
bayesian.classifier


Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
     Kama      Rosa  Canadian 
0.3333333 0.3333333 0.3333333 

Conditional probabilities:
          area
Y              [,1]      [,2]
  Kama     14.48857 1.1679451
  Rosa     18.36245 1.4282325
  Canadian 11.90551 0.7314855

          length
Y              [,1]      [,2]
  Kama     5.522245 0.2315241
  Rosa     6.141102 0.2289450
  Canadian 5.218224 0.1370204

          AC
Y              [,1]     [,2]
  Kama     2.553596 1.143665
  Rosa     3.704714 1.279780
  Canadian 4.930959 1.415836


In [54]:
bayesian.results = predict(object = bayesian.classifier, test.set)
test.set["predicted"] = bayesian.results

In [55]:
#test.set

In [56]:
conf.matrix.cb = confusionMatrix(table(test.set$class, test.set$predicted))
print(conf.matrix.cb)

Confusion Matrix and Statistics

          
           Kama Rosa Canadian
  Kama       16    1        4
  Rosa        2   19        0
  Canadian    1    0       20

Overall Statistics
                                         
               Accuracy : 0.873          
                 95% CI : (0.765, 0.9435)
    No Information Rate : 0.381          
    P-Value [Acc > NIR] : 8.112e-16      
                                         
                  Kappa : 0.8095         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: Kama Class: Rosa Class: Canadian
Sensitivity               0.8421      0.9500          0.8333
Specificity               0.8864      0.9535          0.9744
Pos Pred Value            0.7619      0.9048          0.9524
Neg Pred Value            0.9286      0.9762          0.9048
Prevalence                0.3016      0.3175          0.3810
Detection Rate            0.2540      0.3016     